# Plot Figures

Various figures for visualization only

## Gaussian Mixture Model

Dummy Roadmap for visualizing Gaussian Mixture Model

In [ ]:
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi, voronoi_plot_2d

from st_gaussian_prm.envs import ObstacleMap
from st_gaussian_prm.utils import GaussianPRM

obstacle_map = ObstacleMap(80, 60)
g_map = GaussianPRM(obstacle_map, 10)
g_map.roadmap_construction()

fig, ax = g_map.visualize_map()

vor = Voronoi(g_map.samples)

voronoi_plot_2d(vor, ax)

for g_node in g_map.gaussian_nodes:
    g_node.visualize_gaussian(ax)

plt.show()


## CVT


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Voronoi
from shapely.geometry import Polygon

# Dummy roadmap class (no obstacles)
class DummyRoadmap:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        self.obstacles = []

    def is_point_collision(self, point):
        return False

    def is_geometry_collision(self, geom):
        return False

    def get_bounding_polygon(self):
        return Polygon([
            (0, 0), (self.width, 0), (self.width, self.height), (0, self.height)
        ])

# Compute centroid for each Voronoi cell
def compute_centroids(vor, points, roadmap):
    bounding_polygon = roadmap.get_bounding_polygon()
    new_points = []

    for point, region_index in zip(points, vor.point_region):
        region = vor.regions[region_index]
        if not region or -1 in region:
            new_points.append(point)
            continue

        region_vertices = [vor.vertices[i] for i in region]
        cell_polygon = Polygon(region_vertices)

        if roadmap.is_geometry_collision(cell_polygon):
            new_points.append(point)
        elif cell_polygon.is_valid and not cell_polygon.is_empty:
            centroid = cell_polygon.centroid
            if bounding_polygon.contains(centroid):
                new_points.append(centroid.coords[0])
            else:
                closest_point = bounding_polygon.exterior.interpolate(
                    bounding_polygon.exterior.project(centroid)
                )
                new_points.append(closest_point.coords[0])
        else:
            new_points.append(point)
    return np.array(new_points)

# Main visualization function
def visualize_cvt_step_with_cells(points, roadmap):
    vor_initial = Voronoi(points)
    centroids = compute_centroids(vor_initial, points, roadmap)
    vor_updated = Voronoi(centroids)

    fig, axs = plt.subplots(1, 4, figsize=(15, 5))

    # Panel 1: Initial Voronoi
    ax = axs[0]
    for region_index in vor_initial.regions:
        if not region_index or -1 in region_index:
            continue
        region = [vor_initial.vertices[i] for i in region_index]
        poly = Polygon(region)
        if poly.is_valid and not poly.is_empty:
            x, y = poly.exterior.xy
            ax.fill(x, y, alpha=0.4, facecolor="white", edgecolor='black')
    ax.plot(points[:, 0], points[:, 1], 'ko')
    ax.set_xlim(0, roadmap.width)
    ax.set_ylim(0, roadmap.height)
    ax.set_title("Initial Voronoi")
    ax.set_aspect('equal')
    ax.axis('off')

    # Panel 2: Voronoi Cells + Centroids
    ax = axs[1]
    for region_index in vor_initial.regions:
        if not region_index or -1 in region_index:
            continue
        region = [vor_initial.vertices[i] for i in region_index]
        poly = Polygon(region)
        if poly.is_valid and not poly.is_empty:
            x, y = poly.exterior.xy
            ax.fill(x, y, alpha=0.2, facecolor="white", edgecolor='gray')
    ax.plot(points[:, 0], points[:, 1], 'ko', label='Original')
    ax.plot(centroids[:, 0], centroids[:, 1], 'ro', label='Centroid')
    for p, c in zip(points, centroids):
        ax.plot([p[0], c[0]], [p[1], c[1]], 'k--', linewidth=0.5)
    ax.set_xlim(0, roadmap.width)
    ax.set_ylim(0, roadmap.height)
    ax.set_title("Centroid Update Step")
    ax.set_aspect('equal')
    ax.axis('off')
    ax.legend()

    # Panel 3: Updated Voronoi
    ax = axs[2]
    for region_index in vor_updated.regions:
        if not region_index or -1 in region_index:
            continue
        region = [vor_updated.vertices[i] for i in region_index]
        poly = Polygon(region)
        if poly.is_valid and not poly.is_empty:
            x, y = poly.exterior.xy
            ax.fill(x, y, alpha=0.4, facecolor="white", edgecolor='black')
    ax.plot(centroids[:, 0], centroids[:, 1], 'ro')
    ax.set_xlim(0, roadmap.width)
    ax.set_ylim(0, roadmap.height)
    ax.set_title("Updated Voronoi")
    ax.set_aspect('equal')
    ax.axis('off')

    # Panel 4: Final Voronoi
    ax = axs[3]
    vor = Voronoi(centroids)
    for i in range(10):
        centroids = compute_centroids(vor, centroids, roadmap)
        vor = Voronoi(centroids)
    for region_index in vor.regions:
        if not region_index or -1 in region_index:
            continue
        region = [vor.vertices[i] for i in region_index]
        poly = Polygon(region)
        if poly.is_valid and not poly.is_empty:
            x, y = poly.exterior.xy
            ax.fill(x, y, alpha=0.4, facecolor="white", edgecolor='black')
    ax.plot(centroids[:, 0], centroids[:, 1], 'ko')
    ax.set_xlim(0, roadmap.width)
    ax.set_ylim(0, roadmap.height)
    ax.set_title("Final Voronoi")
    ax.set_aspect('equal')
    ax.axis('off')


    plt.tight_layout()
    plt.show()
    fig.savefig("./solutions/cvt.pdf")

# Run the visualization
if __name__ == "__main__":
    roadmap = DummyRoadmap(100, 100)
    np.random.seed(0)
    init_points = np.random.rand(30, 2) * 100

    visualize_cvt_step_with_cells(init_points, roadmap)


## Visualize Gaussian path

Setting up a simple instance, and then solve it 

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon

In [ ]:
# Instance Config 
instance_config = {
"capacity_percentage" : 0.8,
"agent_radius" : 0.5,
"map_type" : "obstacle" ,
"num_samples" : 100 ,
"start_regions" : [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
],
"goal_regions" : [
    Polygon([[170, 160], [170, 110], [200, 110], [200, 160]]),
    Polygon([[170, 40], [170, 0], [200, 0], [200, 40]]),
],
"num_starts" : 10,
"num_goals" : 10,
}

# Solver Config

solver_config = {
    "solver_name": "TEGSolver"
}

configs = {
    "time_limit": 180,  # 3 mins
    "capacity_constraint": False
}

In [ ]:
from st_gaussian_prm.utils import load_instance

solver = load_instance(instance_config, solver_config, **configs)

gaussian_prm = solver.gaussian_prm
agent_radius = instance_config["agent_radius"]
starts_idx = solver.starts_idx
goals_idx = solver.goals_idx
num_agents = solver.num_agents


In [ ]:
solution = solver.solve()
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
candid_starts_idx = solution["starts_idx"]
candid_goals_idx = solution["goals_idx"]

In [ ]:
from matplotlib.cm import ScalarMappable
import matplotlib.colors as colors

cmap = plt.colormaps['coolwarm']
norm = colors.Normalize(vmin=0, vmax=solution["timestep"]-1) 

fig, ax = gaussian_prm.visualize_g_nodes()

for timestep in range(solution["timestep"]):
    color = cmap(norm(timestep))
    positions = [path[timestep] for path in solution["paths"]]
    g_nodes = [gaussian_prm.gaussian_nodes[pos] for pos in positions]
    for g_node in g_nodes:
        ellipse = g_node.get_confidence_ellipse()
        x, y = ellipse.exterior.xy
        ax.fill(x, y, facecolor=color)

ax.set_xticks([])
ax.set_yticks([])

sm = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])  # Required for plt.colorbar()
cbar = plt.colorbar(sm, ax=ax, orientation='horizontal', pad=0.01 )
cbar.set_label('Timestep')

plt.show()
fig.savefig("./solutions/gaussian_trajectory.pdf", bbox_inches="tight")

## Formation Control

Visualizing the reagions sampled by Formation Control Solver

In [ ]:
# Instance Config 
instance_config = {
"capacity_percentage" : 0.8,
"agent_radius" : 0.5,
"map_type" : "obstacle" ,
"num_samples" : 100 ,
"start_regions" : [
    Polygon([[0, 40], [0, 0], [50, 0], [50, 40]]),
    Polygon([[0, 160], [0, 120], [50, 120], [50, 160]])
],
"goal_regions" : [
    Polygon([[170, 160], [170, 110], [200, 110], [200, 160]]),
    Polygon([[170, 40], [170, 0], [200, 0], [200, 40]]),
],
"num_starts" : 4,
"num_goals" : 5,
}

# Solver Config

solver_config = {
    "solver_name": "FormationControlSolver"
}

configs = {
    "time_limit": 180,  # 3 mins
    "capacity_constraint": False
}

In [ ]:
from st_gaussian_prm.utils import load_instance

solver = load_instance(instance_config, solver_config, **configs)

gaussian_prm = solver.gaussian_prm
agent_radius = instance_config["agent_radius"]
starts_idx = solver.starts_idx
goals_idx = solver.goals_idx
num_agents = solver.num_agents

In [ ]:
solution = solver.solve()
assert solution["success"], "solver failed."
timestep = solution["timestep"]
paths = solution["paths"]
g_nodes = solution["g_nodes"]
candid_starts_idx = solution["starts_idx"]
candid_goals_idx = solution["goals_idx"]

In [ ]:
# plot convex sets
convex_sets = solution["convex_nodes"]
g_nodes = solution["g_nodes"]
fig, ax = gaussian_prm.visualize_map()

for cs in convex_sets:
    x, y = cs.poly.exterior.xy  # get exterior boundary coordinates
    ax.fill(x, y, alpha=0.3, facecolor="grey", edgecolor="black")

for idx in solution["paths"][0][1:-1]:
    g_node = solution["g_nodes"][idx]
    g_node.visualize(ax, edgecolor="blue")

for start in starts_idx:
    gaussian_prm.gaussian_nodes[start].visualize_gaussian(ax, cmap="Reds")

for goal in goals_idx :
    gaussian_prm.gaussian_nodes[goal].visualize_gaussian(ax)
 
ax.set_xticks([])
ax.set_yticks([])
plt.show()
fig.savefig("./solutions/formation_control.pdf", bbox_inches="tight")

## Plot color coded overflow nodes